## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Castro,BR,-24.7911,-50.0119,54.27,41,9,4.27,clear sky
1,1,Saskylakh,RU,71.9167,114.0833,59.31,63,98,7.49,overcast clouds
2,2,Nemuro,JP,43.3236,145.5750,63.05,98,98,5.26,overcast clouds
3,3,Neuquen,AR,-38.9516,-68.0591,42.69,65,100,2.30,overcast clouds
4,4,Faanui,PF,-16.4833,-151.7500,78.30,75,30,23.49,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 72


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Castro,BR,-24.7911,-50.0119,54.27,41,9,4.27,clear sky
1,1,Saskylakh,RU,71.9167,114.0833,59.31,63,98,7.49,overcast clouds
2,2,Nemuro,JP,43.3236,145.5750,63.05,98,98,5.26,overcast clouds
5,5,Busselton,AU,-33.6500,115.3333,54.05,87,100,16.60,light rain
6,6,Rosetown,CA,51.5501,-108.0014,60.78,94,100,10.36,overcast clouds
15,15,Carnarvon,AU,-24.8667,113.6333,69.87,88,75,11.50,broken clouds
20,20,Salto,UY,-31.3833,-57.9667,50.88,65,28,8.99,scattered clouds
23,23,Airai,TL,-8.9266,125.4092,56.95,87,39,3.53,scattered clouds
26,26,Bambous Virieux,MU,-20.3428,57.7575,70.05,83,75,20.71,light intensity shower rain
31,31,Vaini,TO,-21.2000,-175.2000,68.16,94,20,2.30,fog


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             249
City                   249
Country                245
Lat                    249
Lng                    249
Max Temp               249
Humidity               249
Cloudiness             249
Wind Speed             249
Current Description    249
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(0)

clean_df.count()

Unnamed: 0             245
City                   245
Country                245
Lat                    245
Lng                    245
Max Temp               245
Humidity               245
Cloudiness             245
Wind Speed             245
Current Description    245
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.nan
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Castro,BR,54.27,clear sky,-24.7911,-50.0119,NaN
1,Saskylakh,RU,59.31,overcast clouds,71.9167,114.0833,NaN
2,Nemuro,JP,63.05,overcast clouds,43.3236,145.5750,NaN
5,Busselton,AU,54.05,light rain,-33.6500,115.3333,NaN
6,Rosetown,CA,60.78,overcast clouds,51.5501,-108.0014,NaN
15,Carnarvon,AU,69.87,broken clouds,-24.8667,113.6333,NaN
20,Salto,UY,50.88,scattered clouds,-31.3833,-57.9667,NaN
23,Airai,TL,56.95,scattered clouds,-8.9266,125.4092,NaN
26,Bambous Virieux,MU,70.05,light intensity shower rain,-20.3428,57.7575,NaN
31,Vaini,TO,68.16,fog,-21.2000,-175.2000,NaN


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Use the search term: "lodging" and our latitude and longitude.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Castro,BR,54.27,clear sky,-24.7911,-50.0119,CHACARA BAILLY
1,Saskylakh,RU,59.31,overcast clouds,71.9167,114.0833,NaN
2,Nemuro,JP,63.05,overcast clouds,43.3236,145.5750,Hotel Nemuro Kaiyoutei
5,Busselton,AU,54.05,light rain,-33.6500,115.3333,Observatory Guest House
6,Rosetown,CA,60.78,overcast clouds,51.5501,-108.0014,Travelodge by Wyndham Rosetown
15,Carnarvon,AU,69.87,broken clouds,-24.8667,113.6333,Hospitality Carnarvon
20,Salto,UY,50.88,scattered clouds,-31.3833,-57.9667,Gran Hotel Uruguay
23,Airai,TL,56.95,scattered clouds,-8.9266,125.4092,Juvinal Martin
26,Bambous Virieux,MU,70.05,light intensity shower rain,-20.3428,57.7575,Casa Tia Villa
31,Vaini,TO,68.16,fog,-21.2000,-175.2000,Keleti Beach Resort


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Castro,BR,54.27,clear sky,-24.7911,-50.0119,CHACARA BAILLY
2,Nemuro,JP,63.05,overcast clouds,43.3236,145.5750,Hotel Nemuro Kaiyoutei
5,Busselton,AU,54.05,light rain,-33.6500,115.3333,Observatory Guest House
6,Rosetown,CA,60.78,overcast clouds,51.5501,-108.0014,Travelodge by Wyndham Rosetown
15,Carnarvon,AU,69.87,broken clouds,-24.8667,113.6333,Hospitality Carnarvon
20,Salto,UY,50.88,scattered clouds,-31.3833,-57.9667,Gran Hotel Uruguay
23,Airai,TL,56.95,scattered clouds,-8.9266,125.4092,Juvinal Martin
26,Bambous Virieux,MU,70.05,light intensity shower rain,-20.3428,57.7575,Casa Tia Villa
31,Vaini,TO,68.16,fog,-21.2000,-175.2000,Keleti Beach Resort
32,Maningrida,AU,69.39,scattered clouds,-12.0667,134.2667,Dhukurrdji Lodge


In [10]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv("WeatherPy_vacation.csv", index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""


In [12]:

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.6)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))